In [5]:
import torch
from torchvision.datasets import MNIST
from torchvision import transforms
from torch import optim
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader,Dataset
from torch.autograd import Variable

In [6]:
# Build CustomedDataset
# Separate the given dataset into two parts, four for train and one for test

class CustomedDataSet(Dataset):
    def __init__(self, train=True):
        self.train = train
        if self.train :
            trainX = pd.read_csv('./train.csv',nrows=33600)
            trainY = trainX.label.as_matrix().tolist()
            trainX = trainX.drop('label',axis=1).as_matrix().reshape(trainX.shape[0], 1, 28, 28)
            self.datalist = trainX
            self.labellist = trainY
        else:
            testX = pd.read_csv('./train.csv')
            testX = testX.drop(testX.head(33600).index)
            testY = testX.label.as_matrix().tolist()
            testX = testX.drop('label',axis=1).as_matrix().reshape(testX.shape[0], 1, 28, 28)
            self.datalist = testX
            self.labellist = testY
            
    def __getitem__(self, index):
        if self.train:
            return torch.Tensor(self.datalist[index].astype(float)),self.labellist[index]
        else:
            return torch.Tensor(self.datalist[index].astype(float)),self.labellist[index]
    
    def __len__(self):
        return self.datalist.shape[0]

In [7]:
# Hyper parameters
batch_size = 100
learning_rate = 1e-2
epoches = 5

In [8]:
train_dataset = CustomedDataSet()
test_dataset  = CustomedDataSet(train=False)
train_loader  = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True,num_workers=2)
test_loader   = DataLoader(dataset= test_dataset,batch_size=batch_size,shuffle=False)

/home/peng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/home/peng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
/home/peng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]
/home/peng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [9]:
# LeNet model

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv=nn.Sequential(
            nn.Conv2d(1,6,5,stride=1,padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(6,16,5,stride=1,padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(16,120,5,stride=1,padding=2),
            nn.ReLU(),
        )
        self.fc=nn.Sequential(          # Fully connected layer
            nn.Linear(7*7*120,120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84,10),
            nn.Sigmoid(),
        )
    def forward(self, x):
        out=self.conv(x)
        out=out.view(out.size(0),-1)
        out=self.fc(out)
        return out

In [10]:
lenet = LeNet()  # network

In [11]:
criterian = nn.CrossEntropyLoss() 
optimizer = optim.SGD(lenet.parameters(), lr=learning_rate)

In [12]:
for epoch in range(epoches):
    for i, (img,label) in enumerate(train_loader):
        optimizer.zero_grad()
        img = Variable(img)
        label = Variable(label)

        output=lenet(img)
        loss=criterian(output,label)
        loss.backward()
        optimizer.step()
        if (i+1) % 112 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' %(epoch+1, epoches, i+1, 
                                                              len(train_dataset)//batch_size, loss.item()))

Epoch [1/5], Iter [112/336] Loss: 1.5954
Epoch [1/5], Iter [224/336] Loss: 1.5665
Epoch [1/5], Iter [336/336] Loss: 1.5119
Epoch [2/5], Iter [112/336] Loss: 1.5054
Epoch [2/5], Iter [224/336] Loss: 1.5418
Epoch [2/5], Iter [336/336] Loss: 1.5054
Epoch [3/5], Iter [112/336] Loss: 1.4894
Epoch [3/5], Iter [224/336] Loss: 1.4931
Epoch [3/5], Iter [336/336] Loss: 1.4843
Epoch [4/5], Iter [112/336] Loss: 1.4996
Epoch [4/5], Iter [224/336] Loss: 1.5399
Epoch [4/5], Iter [336/336] Loss: 1.4744
Epoch [5/5], Iter [112/336] Loss: 1.4854
Epoch [5/5], Iter [224/336] Loss: 1.4886
Epoch [5/5], Iter [336/336] Loss: 1.4700


In [13]:
ans = torch.LongTensor()

In [14]:
for (images,labels) in test_loader:
    images = Variable(images)
    outputs = lenet(images)
    _,predicted = torch.max(outputs.data, 1)
    ans = torch.cat((ans,predicted),0)

In [15]:
ans = ans.numpy()

In [16]:
corr=0
for i,(images,labels) in enumerate(test_dataset):
    if ans[i]==labels:
        corr=corr+1
print("%.3F"%(corr/84))

97.631
